In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# Gaussian kernel density estimate for density plot
from scipy.stats import gaussian_kde

# List of lists to single list
from itertools import chain

In [2]:
# Bokeh plotting tools
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import (CategoricalColorMapper, HoverTool, ColumnDataSource, Panel, 
                          FuncTickFormatter, SingleIntervalTicker, LinearAxis)
from bokeh.models.widgets import (CheckboxGroup, Slider, RangeSlider, Tabs, CheckboxButtonGroup, 
                                  TableColumn, DataTable, Select)

from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

# Display plots in the notebook
output_notebook()

Loading BokehJS ...

# Data

## Flights and Airlines

In [3]:
flights = pd.read_csv('data/flights.csv', index_col=0)
available_carriers = list(set(flights['name']))

# Sort is an inplace operation and return a none object
available_carriers.sort()
airline_colors = Category20_16
airline_colors.sort()

flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,name
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,United Air Lines Inc.
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,United Air Lines Inc.
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,American Airlines Inc.
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,JetBlue Airways
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,Delta Air Lines Inc.


In [4]:
flights['arr_delay'].describe()

count    327346.000000
mean          6.895377
std          44.633292
min         -86.000000
25%         -17.000000
50%          -5.000000
75%          14.000000
max        1272.000000
Name: arr_delay, dtype: float64

## State Map Data

In [5]:
# Included data in Bokeh for map
from bokeh.sampledata.us_states import data as states

# Remove Alaska and Hawaii
if 'HI' in states: del states['HI']
if 'AK' in states: del states['AK']
    
# Put longitudes and latitudes in lists
xs = [states[state]['lons'] for state in states]
ys = [states[state]['lats'] for state in states]

# Formatted Flight Delay Data for map
map_data = pd.read_csv('data/flights_map.csv', header=[0,1], index_col=0)
map_data.head()

origin               dest                   carrier arr_delay  \
  Unnamed: 1_level_1 Unnamed: 2_level_1        Unnamed: 3_level_1     count   
0                EWR                ALB  ExpressJet Airlines Inc.       418   
2                EWR                ATL         Endeavor Air Inc.         4   
3                EWR                ATL      Delta Air Lines Inc.      3116   
4                EWR                ATL  ExpressJet Airlines Inc.      1654   
5                EWR                ATL     United Air Lines Inc.       102   

                          air_time                            ...          \
        mean   min    max    count        mean    min         ...           
0  14.397129 -34.0  328.0      418   31.787081   24.0         ...           
2  -6.250000 -24.0   15.0        4  111.000000  105.0         ...           
3   9.996470 -38.0  796.0     3116  110.853659   88.0         ...           
4  19.546554 -39.0  454.0     1654  114.042322   90.0         ...           
5  10.500000 -37.0  360.0      102  113.647059   97.0         ...           

  distance      air_speed (mph)                                      \
       min  max           count        mean         min         max   
0      143  143             418  272.300976  171.600000  357.500000   
2      746  746               4  403.899553  382.564103  426.285714   
3      746  746            3116  406.542093  254.318182  508.636364   
4      746  746            1654  395.945542  278.012422  497.333333   
5      746  746             102  396.577226  315.211268  461.443299   

           start_long          start_lati            end_long  \
  Unnamed: 20_level_1 Unnamed: 21_level_1 Unnamed: 22_level_1   
0          -74.168701           40.692501          -73.801697   
2          -74.168701           40.692501          -84.428101   
3          -74.168701           40.692501          -84.428101   
4          -74.168701           40.692501          -84.428101   
5          -74.168701           40.692501          -84.428101   

             end_lati  
  Unnamed: 23_level_1  
0           42.748299  
2           33.636700  
3           33.636700  
4           33.636700  
5           33.636700  

[5 rows x 23 columns]

# Map of Flight Delays

In [14]:
def modify_doc(doc):
    
    # Function to make a dataset for the map based on a list of carriers
    def make_dataset(carrier_list):
        
        # Subset to the carriers in the specified list
        subset = map_data[map_data['carrier']['Unnamed: 3_level_1'].isin(carrier_list)]

        
        # Dictionary mapping carriers to colors
        color_dict = {carrier: color for carrier, color in zip(list(set(map_data['carrier']['Unnamed: 3_level_1'])), 
                                                               Category20_16)}
        
        # Lists of data for plotting
        flight_x = []
        flight_y = []
        colors = []
        carriers = []
        counts = []
        mean_delays = []
        min_delays = []
        max_delays = []
        dest_loc = []
        origin_x_loc = []
        origin_y_loc = []
        dest_x_loc = []
        dest_y_loc = []
        origins = []
        dests = []
        distances = []

        # Iterate through each carrier
        for carrier in carrier_list:

            # Subset to the carrier
            sub_carrier = subset[subset['carrier']['Unnamed: 3_level_1'] == carrier]

            # Iterate through each route (origin to destination) for the carrier
            for _, row in sub_carrier.iterrows():

                colors.append(color_dict[carrier])
                carriers.append(carrier)
                origins.append(row['origin']['Unnamed: 1_level_1'])
                dests.append(row['dest']['Unnamed: 2_level_1'])

                # Origin x (longitude) and y (latitude) location
                origin_x_loc.append(row['start_long']['Unnamed: 20_level_1'])
                origin_y_loc.append(row['start_lati']['Unnamed: 21_level_1'])

                # Destination x (longitude) and y latitude (location)
                dest_x_loc.append(row['end_long']['Unnamed: 22_level_1'])
                dest_y_loc.append(row['end_lati']['Unnamed: 23_level_1'])

                # Flight x (longitude) locations
                flight_x.append([row['start_long']['Unnamed: 20_level_1'], 
                                 row['end_long']['Unnamed: 22_level_1']])

                # Flight y (latitude) locations
                flight_y.append([row['start_lati']['Unnamed: 21_level_1'], 
                                 row['end_lati']['Unnamed: 23_level_1']])


                # Stats about the particular route
                counts.append(row['arr_delay']['count'])
                mean_delays.append(row['arr_delay']['mean'])
                min_delays.append(row['arr_delay']['min'])
                max_delays.append(row['arr_delay']['max'])
                distances.append(row['distance']['mean'])


        # Create a column data source from the lists of lists
        new_map_src = ColumnDataSource(data = {'carrier': carriers, 'flight_x': flight_x, 'flight_y': flight_y, 
                                               'origin_x_loc': origin_x_loc, 'origin_y_loc': origin_y_loc,
                                               'dest_x_loc': dest_x_loc, 'dest_y_loc': dest_y_loc,
                                               'color': colors, 'count': counts, 'mean_delay': mean_delays,
                                               'origin': origins, 'dest': dests, 'distance': distances,
                                               'min_delay': min_delays, 'max_delay': max_delays})

        return new_map_src

    def make_plot(map_src):
        
        # Create the plot with no axes or grid
        p = figure(plot_width = 1100, plot_height = 700, title = 'Map of 2013 Flight Delays Departing NYC')
        p.xaxis.visible = False
        p.yaxis.visible = False
        p.grid.visible = False

        # States are drawn as patches
        patches_glyph = p.patches(xs, ys, fill_alpha=0.2, fill_color = 'lightgray', 
                                  line_color="#884444", line_width=2, line_alpha=0.8)

        # Airline flights are drawn as lines
        lines_glyph = p.multi_line('flight_x', 'flight_y', color = 'color', line_width = 2, 
                                   line_alpha = 0.8, hover_line_alpha = 1.0, hover_line_color = 'color',
                                   legend = 'carrier', source = map_src)

        # Origins are drawn as squares (all in NYC)
        squares_glyph = p.square('origin_x_loc', 'origin_y_loc', color = 'color', size = 10, source = map_src, 
                                 legend = 'carrier')

        # Destinations are drawn as circles
        circles_glyph = p.circle('dest_x_loc', 'dest_y_loc', color = 'color', size = 10, source = map_src, 
                                 legend = 'carrier')

        # Add the glyphs to the plot using the renderers attribute
        p.renderers.append(patches_glyph)
        p.renderers.append(lines_glyph)
        p.renderers.append(squares_glyph)
        p.renderers.append(circles_glyph)
        
        # Hover tooltip for flight lines, assign only the l
        hover_line = HoverTool(tooltips=[('Airline', '@carrier'),
                                    ('Number of Flights', '@count'),
                                    ('Average Delay', '@mean_delay{0.0}'),
                                    ('Max Delay', '@max_delay{0.0}'),
                                    ('Min Delay', '@min_delay{0.0}')],
                              line_policy = 'next',
                              renderers = [lines_glyph])
        
        # Hover tooltip for origin and destination
        hover_circle = HoverTool(tooltips=[('Origin', '@origin'),
                                           ('Dest', '@dest'),
                                           ('Distance (miles)', '@distance')],
                                renderers = [circles_glyph])

        # Position the location so it does not overlap plot
        p.legend.location = (10, 50)

        # Add the hovertools to the figure
        p.add_tools(hover_line)
        p.add_tools(hover_circle)

        p = style(p) 
        
        return p
    
    # Styling 
    def style(p):
            
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p
        
    # Show selected carriers on map
    def update(attr, old, new):
        # Find list of carriers and make a new data set
        carrier_list = [carrier_selection.labels[i] for i in carrier_selection.active]
        new_map_src = make_dataset(carrier_list)

        map_src.data.update(new_map_src.data)
            
            
            
    # CheckboxGroup to select carriers for plotting    
    carrier_selection = CheckboxGroup(labels=available_carriers, active = [0, 1])
    carrier_selection.on_change('active', update)

    # Initial carriers to plot
    initial_carriers = [carrier_selection.labels[i] for i in carrier_selection.active]

    # Initial source and plot
    map_src = make_dataset(initial_carriers)

    p = make_plot(map_src)

    # Layout setup
    layout = row(carrier_selection, p)
    tab = Panel(child = layout, title = 'Flight Map')

    tabs = Tabs(tabs=[tab])
    doc.add_root(tabs)
        
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [15]:
show(app)

# Histogram

In [8]:
def modify_doc(doc):
    
    # Function to make a dataset for histogram based on a list of carriers
    # a minimum delay, maximum delay, and histogram bin width
    def make_dataset(carrier_list, range_start = -60, range_end = 120, bin_width = 5):

        # Dataframe to hold information
        by_carrier = pd.DataFrame(columns=['proportion', 'left', 'right', 
                                           'f_proportion', 'f_interval',
                                           'name', 'color'])
        
        range_extent = range_end - range_start

        # Iterate through all the carriers
        for i, carrier_name in enumerate(carrier_list):

            # Subset to the carrier
            subset = flights[flights['name'] == carrier_name]

            # Create a histogram with 5 minute bins
            arr_hist, edges = np.histogram(subset['arr_delay'], 
                                           bins = int(range_extent / bin_width), 
                                           range = [range_start, range_end])

            # Divide the counts by the total to get a proportion
            arr_df = pd.DataFrame({'proportion': arr_hist / np.sum(arr_hist), 'left': edges[:-1], 'right': edges[1:] })

            # Format the proportion 
            arr_df['f_proportion'] = ['%0.5f' % proportion for proportion in arr_df['proportion']]

            # Format the interval
            arr_df['f_interval'] = ['%d to %d minutes' % (left, right) for left, right in zip(arr_df['left'], arr_df['right'])]

            # Assign the carrier for labels
            arr_df['name'] = carrier_name

            # Color each carrier differently
            arr_df['color'] = Category20_16[i]

            # Add to the overall dataframe
            by_carrier = by_carrier.append(arr_df)

        # Overall dataframe
        by_carrier = by_carrier.sort_values(['name', 'left'])

        return ColumnDataSource(by_carrier)

    def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p
    
    def make_plot(src):
        # Blank plot with correct labels
        p = figure(plot_width = 700, plot_height = 700, 
                  title = 'Histogram of Arrival Delays by Airline',
                  x_axis_label = 'Delay (min)', y_axis_label = 'Proportion')

        # Quad glyphs to create a histogram
        p.quad(source = src, bottom = 0, top = 'proportion', left = 'left', right = 'right',
               color = 'color', fill_alpha = 0.7, hover_fill_color = 'color', legend = 'name',
               hover_fill_alpha = 1.0, line_color = 'black')

        # Hover tool with vline mode
        hover = HoverTool(tooltips=[('Carrier', '@name'), 
                                    ('Delay', '@f_interval'),
                                    ('Proportion', '@f_proportion')],
                          mode='vline')

        p.add_tools(hover)

        # Styling
        p = style(p)

        return p
    
    
    
    def update(attr, old, new):
        carriers_to_plot = [carrier_selection.labels[i] for i in carrier_selection.active]
        
        new_src = make_dataset(carriers_to_plot,
                               range_start = range_select.value[0],
                               range_end = range_select.value[1],
                               bin_width = binwidth_select.value)
        
        

        src.data.update(new_src.data)
        
        
    carrier_selection = CheckboxGroup(labels=available_carriers, active = [0, 1])
    carrier_selection.on_change('active', update)
    
    binwidth_select = Slider(start = 1, end = 30, 
                         step = 1, value = 5,
                         title = 'Delay Width (min)')
    binwidth_select.on_change('value', update)
    
    range_select = RangeSlider(start = -60, end = 180, value = (-60, 120),
                               step = 5, title = 'Delay Range (min)')
    range_select.on_change('value', update)
    


    initial_carriers = [carrier_selection.labels[i] for i in carrier_selection.active]
    
    src = make_dataset(initial_carriers,
                      range_start = range_select.value[0],
                      range_end = range_select.value[1],
                      bin_width = binwidth_select.value)

    
    p = make_plot(src)
    
    # Put controls in a single element
    controls = WidgetBox(carrier_selection, binwidth_select, range_select)
    
    # Create a row layout
    layout = row(controls, p)
    
    # Make a tab with the layout 
    tab = Panel(child=layout, title = 'Histogram')
    tabs = Tabs(tabs=[tab])
    
    doc.add_root(tabs)
    
# Set up an application
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [9]:
show(app)

In [10]:
def modify_doc(doc):
    
    def make_kde_dataset(carrier_list, range_start, range_end, bandwidth):

        xs = []
        ys = []
        colors = []
        labels = []

        for i, carrier in enumerate(carrier_list):
            subset = flights[flights['name'] == carrier]
            subset = subset[subset['arr_delay'].between(range_start, range_end)]

            kde = gaussian_kde(subset['arr_delay'], bw_method=bandwidth)
            # Evenly space x values
            x = np.linspace(range_start, range_end, 100)
            # Evaluate pdf at every value of x
            y = kde.pdf(x)

            # Append the values to plot
            xs.append(list(x))
            ys.append(list(y))

            # Append the colors and label
            colors.append(airline_colors[i])
            labels.append(carrier)

        kernel_source = ColumnDataSource(data={'x': xs, 'y': ys, 'color': colors, 'label': labels})

        return kernel_source

    def make_kde_plot(kde_src):
        p = figure(plot_width = 700, plot_height = 700,
                   title = 'Density Plot of Arrival Delays by Airline',
                   x_axis_label = 'Delay (min)', y_axis_label = 'Density')


        p.multi_line('x', 'y', color = 'color', legend = 'label', 
                     line_width = 3,
                     source = kde_src)

        # Hover tool with next line policy
        hover = HoverTool(tooltips=[('Carrier', '@label'), 
                                    ('Delay', '$x'),
                                    ('Density', '$y')],
                          line_policy = 'next')

        # Add the hover tool and styling
        p.add_tools(hover)

        p = style(p)

        return p
    
    def update(attr, old, new):
        # List of carriers to plot
        carriers_to_plot = [carrier_selection.labels[i] for i in carrier_selection.active]
        
        # If no bandwidth is selected, use the default value
        if bandwidth_choose.active == []:
            bandwidth = None
        # If the bandwidth select is activated, use the specified bandwith
        else:
            bandwidth = bandwidth_select.value
            
        
        new_kde_src = make_kde_dataset(carriers_to_plot,
                                       range_start = range_select.value[0],
                                       range_end = range_select.value[1],
                                       bandwidth = bandwidth)
        
        kde_src.data.update(new_kde_src.data)
        
    def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p
    
    # Carriers to plot
    carrier_selection = CheckboxGroup(labels=available_carriers, active = [0, 1])
    carrier_selection.on_change('active', update)
    
    # Range to plot
    range_select = RangeSlider(start = -60, end = 180, value = (-60, 120),
                               step = 5, title = 'Delay Range (min)')
    range_select.on_change('value', update)
    
    # Bandwidth of kernel
    bandwidth_select = Slider(start = 0.1, end = 5, 
                         step = 0.1, value = 0.5,
                         title = 'Bandwidth for Density Plot')
    bandwidth_select.on_change('value', update)
    
    # Whether to set the bandwidth or have it done automatically
    bandwidth_choose = CheckboxButtonGroup(labels=['Choose Bandwidth (Else Auto)'], active = [])
    bandwidth_choose.on_change('active', update)

    initial_carriers = [carrier_selection.labels[i] for i in carrier_selection.active]

    # Make the density data source
    kde_src = make_kde_dataset(initial_carriers, 
                               range_start = range_select.value[0],
                               range_end = range_select.value[1],
                               bandwidth = bandwidth_select.value) 
    
    # Make the density plot
    kde_p = make_kde_plot(kde_src)
    
    # Add style to the plot
    kde_p = style(kde_p)
    
    # Put controls in a single element
    controls = WidgetBox(carrier_selection, range_select, 
                         bandwidth_select, bandwidth_choose)
    
    # Create a row layout
    layout = row(controls, kde_p)
    
    # Make a tab with the layout 
    tab = Panel(child=layout, title = 'Density Plot')
    tabs = Tabs(tabs=[tab])
    
    doc.add_root(tabs)
    
# Set up an application
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [11]:
show(app)

# Table by Airline

In [18]:
# Draw a table with summary statistics by airline
def modify_doc(doc):
    
    carrier_stats = flights.groupby('name')['arr_delay'].describe()
    carrier_stats = carrier_stats.reset_index().rename(columns={'name': 'airline', 
                                                                'count': 'flights', 
                                                                '50%':'median'})
    carrier_stats['mean'] = carrier_stats['mean'].round(2)

    carrier_src = ColumnDataSource(carrier_stats)

    table_columns = [TableColumn(field='airline', title='Airline'),
                     TableColumn(field='flights', title='Number of Flights'),
                     TableColumn(field='min', title='Min Delay'),
                     TableColumn(field='mean', title='Mean Delay'),
                     TableColumn(field='median', title='Median Delay'),
                     TableColumn(field='max', title='Max Delay')]

    carrier_table = DataTable(source=carrier_src, columns=table_columns, width=1000)

    tab = Panel(child = carrier_table, title = 'Summary Table')
    tabs = Tabs(tabs = [tab])
    
    doc.add_root(tabs)
    
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [19]:
show(app)

# Planning

In [30]:
def modify_doc(doc):
    
    def make_dataset(origin, destination):
        # Subset to the selected route
        subset = flights[(flights['dest'] == destination) & (flights['origin'] == origin)]
        
        # Find the carriers who cover particular route
        carriers = list(set(subset['name']))

        # x is the delay, y is the airline
        xs = []
        ys = []
        label_dict = {}
        
        # Iterate through the unique carriers
        for i, carrier in enumerate(carriers):
            
            # Subset to the carrier
            carrier_data = flights[flights['name'] == carrier]
            
            # Append the index of the carrier as many times as there are flights
            # Append the delays for the carrier
            ys.append([i for _ in range(len(carrier_data))])
            xs.append(list(carrier_data['arr_delay']))
  
            # Map the index to the carrier
            label_dict[i]= carrier
            
        xs = list(chain(*xs))
        ys = list(chain(*ys))
            
        new_plan_src = ColumnDataSource(data = {'x': xs, 'y': ys})
        
        return new_plan_src, label_dict
    
    
    def make_plot(src, origin, destination, label_dict):
        
        p = figure(plot_width = 600, plot_height = 200, x_axis_label = 'Delay (min)',
                y_axis_label = '',
                title = 'Arrival Delays for Flights from %s to %s' % (origin, destination))

        p.circle('x', 'y', source = src, alpha = 0.4,
                 color = 'navy', size = 15)
        p.yaxis[0].ticker.desired_num_ticks = len(label_dict)

        p.yaxis.formatter = FuncTickFormatter(code = """
                            var labels = %s;
                            return labels[tick];
                            """ % label_dict)
        
        return p
    
    def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p
    
    def update(attr, old, new):
        # Origin and destination determine values displayed
        origin = origin_select.value
        destination = dest_select.value
        
        # Get the new dataset
        new_plan_src, label_dict = make_dataset(origin, destination)
        
        if len(label_dict) == 0:
            p.title.text = 'No Flights on Record from %s to %s' % (origin, destination)
        
        else:
            p.yaxis[0].ticker.desired_num_ticks = len(label_dict)
            p.yaxis.formatter = FuncTickFormatter(code = """
                                var labels = %s;
                                return labels[tick];
                                """ % label_dict)

            p.title.text = 'Arrival Delays for Flights from %s to %s' % (origin, destination)

        plan_src.data.update(new_plan_src.data)
    
    origins = list(set(flights['origin']))
    dests = list(set(flights['dest']))

    
    origin_select = Select(title = 'Origin', value = 'JFK', options = origins)
    origin_select.on_change('value', update)

    dest_select = Select(title = 'Destination', value = 'BQN', options = dests)
    dest_select.on_change('value', update)
    
    initial_origin = origin_select.value
    initial_dest = dest_select.value
    
    plan_src, label_dict = make_dataset(initial_origin, initial_dest)
    
    p = make_plot(plan_src, initial_origin, initial_dest, label_dict)
    p = style(p)
    
    controls = WidgetBox(origin_select, dest_select)
    layout = row(controls, p)
    tab = Panel(child = layout, title = 'Route Details')
    tabs = Tabs(tabs = [tab])
    doc.add_root(tabs)

handler = FunctionHandler(modify_doc)
app = Application(handler)

In [31]:
show(app)

In [ ]:
}